In [7]:
import sys 
sys.path.insert(1, "../")
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import ast

import dask.dataframe as dd
from dask.dataframe import from_pandas

from workloads.util import use_results, use_dataset, join_queries_features

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
experiment = "stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000"

results_dir = use_results(experiment)
dataset_dir = use_dataset(experiment, redownload=True)
print(results_dir)
print(dataset_dir)

/data/wooders/ralf-vldb//datasets/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000
/data/wooders/ralf-vldb//results/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000
/data/wooders/ralf-vldb//datasets/stl-yahoo-A4-keys-100-interval-10000-events-200000-queries-200000


In [3]:
events_df = pd.read_csv(f"{dataset_dir}/events.csv")
oracle_df = pd.read_csv(f"{dataset_dir}/oracle_features_672.csv").dropna()
oracle_df["seasonality"] = oracle_df['seasonality'].dropna().apply(ast.literal_eval)

In [164]:
queries_df = pd.read_csv(f"{dataset_dir}/queries.csv").set_index("query_id", drop=False)

## Check Single Result 

In [165]:
def join_data(result_df, timestamp_df, queries_df, oracle_df):
    start_ts = timestamp_df.timestamp.min()
    
    queries_df = queries_df.merge(timestamp_df, on="timestamp_ms").set_index("query_id", drop=False)
    #queries_df.timestamp = queries_df.timestamp - start_ts
    
    oracle_df = oracle_df.merge(timestamp_df, on="timestamp_ms")
    #oracle_df.timestamp = oracle_df.timestamp - start_ts
    
    joined_oracle_df = queries_df.merge(oracle_df, on=["timestamp_ms", "key_id", "value"]).set_index("query_id", drop=False)
    joined_df = join_queries_features(queries_df, result_df).set_index("query_id", drop=False)
    
    return joined_oracle_df, joined_df

In [166]:
result_file = "results_workers_4_lifo_window_672_slide_48.csv"
timestamp_file = "results_workers_4_lifo_window_672_slide_48_timestamps.csv"
result_df = pd.read_csv(f"{results_dir}/{result_file}")
result_df['trend'] = result_df['trend'].dropna().apply(ast.literal_eval)
result_df['seasonality'] = result_df['seasonality'].dropna().apply(ast.literal_eval)
timestamp_df = pd.read_csv(f"{results_dir}/{timestamp_file}")

In [167]:
oracle_features_df, features_df = join_data(result_df, timestamp_df, queries_df, oracle_df)

100%|██████████| 194600/194600 [02:38<00:00, 1225.44it/s]


In [168]:
from workloads.stl.stl_util import predict, predict_seasonality
from tqdm import tqdm 

In [169]:
def evaluate(features_df, oracle_features_df, queries_df): 

    predictions = [] 
    oracle_predictions = [] 
    values = [] 

    for query_id in tqdm(features_df.index): 
        #print(query_id)
        of = oracle_features_df.loc[query_id]
        f = features_df.loc[query_id]
        q = queries_df.loc[query_id]

        #print(features_df.loc[query_id].trend)

        feature_prediction = predict(
            float(q.value), 
            float(f.trend[-1]), 
            f.seasonality, 
            int(q.timestamp_ms), 
            int(f.timestamp_ms), 
            interval=5
        )

        oracle_prediction = predict(
            q.value, 
            of.trend, 
            of.seasonality, 
            q.timestamp_ms, 
            of.timestamp_ms, 
            interval=5
        )

        assert of.timestamp_ms == q.timestamp_ms
        assert of.value == q.value 

        #print(query_id, "staleness", q.timestamp_ms - f.timestamp_ms)
        #print(feature_prediction, oracle_prediction)

        feature_prediction_seasonality = predict_seasonality(
            f.seasonality, 
            q.timestamp_ms, 
            of.timestamp_ms, 
            interval=5
        )

        oracle_prediction_seasonality = predict_seasonality(
            of.seasonality, 
            q.timestamp_ms, 
            of.timestamp_ms, 
            interval=5
        )

        #print("seasonal", feature_prediction_seasonality, oracle_prediction_seasonality)

        predictions.append(feature_prediction)
        oracle_predictions.append(oracle_prediction)
        values.append(q.value)

    features_df["predictions"] = predictions
    features_df["oracle_predictions"] = oracle_predictions
    features_df["value"] = values
    return features_df 
    

In [170]:
prediction_results_df = evaluate(features_df, oracle_features_df, queries_df)

100%|██████████| 62372/62372 [00:29<00:00, 2109.95it/s]


In [171]:
prediction_results_df

,key_id,trend,seasonality,timestamp_ms,processing_time,runtime,ingest_time,timestamp,query_id,query_key_id,query_timestamp_ms,query_timestamp,predictions,oracle_predictions,value
query_id,,,,,,,,,,,,,,,
1311,1,"[196.46980257191117, 178.14672291171956, 160.8...","[50.62005366343661, -23.49503650075765, 93.605...",6540,1.645404e+09,0.106391,1.645404e+09,1.645404e+09,1311,1.0,6555.0,1.645404e+09,168.955529,-5.301617,3973.386586
1312,1,"[196.46980257191117, 178.14672291171956, 160.8...","[50.62005366343661, -23.49503650075765, 93.605...",6540,1.645404e+09,0.106391,1.645404e+09,1.645404e+09,1312,1.0,6560.0,1.645404e+09,348.806642,2.898617,3933.690724
1313,1,"[196.46980257191117, 178.14672291171956, 160.8...","[50.62005366343661, -23.49503650075765, 93.605...",6540,1.645404e+09,0.106391,1.645404e+09,1.645404e+09,1313,1.0,6565.0,1.645404e+09,430.354540,5.717293,3857.409683
1314,1,"[196.46980257191117, 178.14672291171956, 160.8...","[50.62005366343661, -23.49503650075765, 93.605...",6540,1.645404e+09,0.106391,1.645404e+09,1.645404e+09,1314,1.0,6570.0,1.645404e+09,224.417345,1.588930,3719.152805
1315,1,"[196.46980257191117, 178.14672291171956, 160.8...","[50.62005366343661, -23.49503650075765, 93.605...",6540,1.645404e+09,0.106391,1.645404e+09,1.645404e+09,1315,1.0,6575.0,1.645404e+09,-87.415547,-2.587467,3623.571888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199941,100,"[1495.0196515944103, 1504.7833340779011, 1514....","[105.2727994765928, 77.14365803434406, -9.5747...",9155,1.645404e+09,0.099647,1.645404e+09,1.645404e+09,199941,100.0,9705.0,1.645404e+09,287.758396,128.367513,2887.280075
199942,100,"[1495.0196515944103, 1504.7833340779011, 1514....","[105.2727994765928, 77.14365803434406, -9.5747...",9155,1.645404e+09,0.099647,1.645404e+09,1.645404e+09,199942,100.0,9710.0,1.645404e+09,449.342939,108.185147,2916.348186
199943,100,"[1495.0196515944103, 1504.7833340779011, 1514....","[105.2727994765928, 77.14365803434406, -9.5747...",9155,1.645404e+09,0.099647,1.645404e+09,1.645404e+09,199943,100.0,9715.0,1.645404e+09,278.774386,128.409236,3029.815277


In [172]:
from sktime.performance_metrics.forecasting import mean_squared_scaled_error

In [173]:
key_loss = prediction_results_df.groupby("key_id").apply(
    lambda x: mean_squared_scaled_error(
        y_true=x.predictions.to_numpy(), 
        y_pred=x.oracle_predictions.to_numpy(),
        y_train=x.value.to_numpy(),
    )
)

In [174]:
key_loss.sum() 

639.8986772299805

In [175]:
len(prediction_results_df.index)

62372

## Evaluate and Plot Multiple Results 